___

<a href='http://www.pieriandata.com'><img src='../Pierian_Data_Logo.png'/></a>
___
<center><em>Copyright by Pierian Data Inc.</em></center>
<center><em>For more information, visit us at <a href='http://www.pieriandata.com'>www.pieriandata.com</a></em></center>

# Linear Regression Project Exercise 

Now that we have learned about feature engineering, cross validation, and grid search, let's test all your new skills with a project exercise in Machine Learning. This exercise will have a more guided approach, later on the ML projects will begin to be more open-ended. We'll start off with using the final version of the Ames Housing dataset we worked on through the feature engineering section of the course. Your goal will be to create a Linear Regression Model, train it on the data with the optimal parameters using a grid search, and then evaluate the model's capabilities on a test set.

---
---
---
## Complete the tasks in bold

**TASK: Run the cells under the Imports and Data section to make sure you have imported the correct general libraries as well as the correct datasets. Later on you may need to run further imports from scikit-learn.**

### Imports

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Data

In [32]:
df = pd.read_csv("AMES_Final_DF.csv")

In [33]:
df.head()

,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,...,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD,Sale Condition_AdjLand,Sale Condition_Alloca,Sale Condition_Family,Sale Condition_Normal,Sale Condition_Partial
0,141.0,31770,6,5,1960,1960,112.0,639.0,0.0,441.0,...,0,0,0,0,1,0,0,0,1,0
1,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,270.0,...,0,0,0,0,1,0,0,0,1,0
2,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,406.0,...,0,0,0,0,1,0,0,0,1,0
3,93.0,11160,7,5,1968,1968,0.0,1065.0,0.0,1045.0,...,0,0,0,0,1,0,0,0,1,0
4,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,137.0,...,0,0,0,0,1,0,0,0,1,0


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2925 entries, 0 to 2924
Columns: 274 entries, Lot Frontage to Sale Condition_Partial
dtypes: float64(11), int64(263)
memory usage: 6.1 MB


**TASK: The label we are trying to predict is the SalePrice column. Separate out the data into X features and y labels**

In [35]:
X = df.drop('SalePrice', axis = 1)
y = df['SalePrice']

**TASK: Use scikit-learn to split up X and y into a training set and test set. Since we will later be using a Grid Search strategy, set your test proportion to 10%. To get the same data split as the solutions notebook, you can specify random_state = 101**

In [36]:
from sklearn.model_selection import train_test_split

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=101)

**TASK: The dataset features has a variety of scales and units. For optimal regression performance, scale the X features. Take carefuly note of what to use for .fit() vs what to use for .transform()**

In [38]:
from sklearn.preprocessing import StandardScaler

In [39]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

**TASK: We will use an Elastic Net model. Create an instance of default ElasticNet model with scikit-learn**

In [40]:
from sklearn.linear_model import ElasticNet

In [41]:
elastic_model = ElasticNet()

**TASK:  The Elastic Net model has two main parameters, alpha and the L1 ratio. Create a dictionary parameter grid of values for the ElasticNet. Feel free to play around with these values, keep in mind, you may not match up exactly with the solution choices**

In [42]:
param_grid = {'alpha':[0.1,1,5,10,50,100],
              'l1_ratio':[.1, .5, .7, .9, .95, .99, 1]}

**TASK: Using scikit-learn create a GridSearchCV object and run a grid search for the best parameters for your model based on your scaled training data. [In case you are curious about the warnings you may recieve for certain parameter combinations](https://stackoverflow.com/questions/20681864/lasso-on-sklearn-does-not-converge)**

In [43]:
from sklearn.model_selection import GridSearchCV

In [44]:
help(GridSearchCV)

Help on class GridSearchCV in module sklearn.model_selection._search:

class GridSearchCV(BaseSearchCV)
 |  GridSearchCV(estimator, param_grid, *, scoring=None, n_jobs=None, refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)
 |  
 |  Exhaustive search over specified parameter values for an estimator.
 |  
 |  Important members are fit, predict.
 |  
 |  GridSearchCV implements a "fit" and a "score" method.
 |  It also implements "score_samples", "predict", "predict_proba",
 |  "decision_function", "transform" and "inverse_transform" if they are
 |  implemented in the estimator used.
 |  
 |  The parameters of the estimator used to apply these methods are optimized
 |  by cross-validated grid-search over a parameter grid.
 |  
 |  Read more in the :ref:`User Guide <grid_search>`.
 |  
 |  Parameters
 |  ----------
 |  estimator : estimator object.
 |      This is assumed to implement the scikit-learn estimator interface.
 |      Either es

In [45]:
grid_model = GridSearchCV(estimator = elastic_model,
                           param_grid = param_grid,
                           scoring = 'neg_mean_squared_error',
                            cv = 5, 
                            verbose = 1)

In [46]:
grid_model.fit(X_train, y_train)

Fitting 5 folds for each of 42 candidates, totalling 210 fits


/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 139422008253.34827, tolerance: 1355206692.5276783
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 165405536738.1427, tolerance: 1307913805.6588454
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 132401459345.10944, tolerance: 1415056940.006107
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWa

GridSearchCV(cv=5, estimator=ElasticNet(),
             param_grid={'alpha': [0.1, 1, 5, 10, 50, 100],
                         'l1_ratio': [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]},
             scoring='neg_mean_squared_error', verbose=1)

**TASK: Display the best combination of parameters for your model**

In [47]:
grid_model.best_params_

{'alpha': 100, 'l1_ratio': 1}

In [48]:
pd.DataFrame(grid_model.cv_results_).sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
41,0.151498,0.009712,0.000836,0.000239,100,1,"{'alpha': 100, 'l1_ratio': 1}",-6.527522e+08,-5.774424e+08,-3.762401e+08,-3.950646e+08,-5.724653e+08,-5.147929e+08,1.093760e+08,1
34,0.277506,0.017718,0.000624,0.000210,50,1,"{'alpha': 50, 'l1_ratio': 1}",-6.534345e+08,-5.766999e+08,-3.806750e+08,-4.047287e+08,-5.806001e+08,-5.192277e+08,1.071335e+08,2
12,0.366648,0.027361,0.000544,0.000046,1,0.99,"{'alpha': 1, 'l1_ratio': 0.99}",-6.669618e+08,-5.879018e+08,-3.819024e+08,-4.093708e+08,-5.880459e+08,-5.268365e+08,1.112787e+08,3
3,0.511430,0.010577,0.001145,0.000605,0.1,0.9,"{'alpha': 0.1, 'l1_ratio': 0.9}",-6.672427e+08,-5.881731e+08,-3.820273e+08,-4.096316e+08,-5.882451e+08,-5.270640e+08,1.113135e+08,4
4,0.394886,0.046193,0.001035,0.000219,0.1,0.95,"{'alpha': 0.1, 'l1_ratio': 0.95}",-6.671344e+08,-5.894025e+08,-3.827143e+08,-4.124198e+08,-5.896326e+08,-5.282607e+08,1.108094e+08,5
2,0.331067,0.037948,0.000839,0.000225,0.1,0.7,"{'alpha': 0.1, 'l1_ratio': 0.7}",-6.746170e+08,-5.953424e+08,-3.807417e+08,-4.065177e+08,-5.869686e+08,-5.288375e+08,1.148463e+08,6
27,0.332034,0.003232,0.000903,0.000138,10,1,"{'alpha': 10, 'l1_ratio': 1}",-6.672342e+08,-5.937499e+08,-3.809313e+08,-4.127535e+08,-5.900095e+08,-5.289357e+08,1.117685e+08,7
20,0.334987,0.003751,0.000845,0.000088,5,1,"{'alpha': 5, 'l1_ratio': 1}",-6.714998e+08,-5.966434e+08,-3.804620e+08,-4.191734e+08,-5.913751e+08,-5.318307e+08,1.121230e+08,8
5,0.340548,0.004049,0.001226,0.000710,0.1,0.99,"{'alpha': 0.1, 'l1_ratio': 0.99}",-6.726844e+08,-5.963010e+08,-3.823734e+08,-4.166526e+08,-5.921745e+08,-5.320372e+08,1.124641e+08,9
19,0.108382,0.003851,0.000754,0.000230,5,0.99,"{'alpha': 5, 'l1_ratio': 0.99}",-6.815304e+08,-6.063142e+08,-3.802629e+08,-4.062500e+08,-5.868648e+08,-5.322445e+08,1.180923e+08,10


**TASK: Evaluate your model's performance on the unseen 10% scaled test set. In the solutions notebook we achieved an MAE of $\$$14149 and a RMSE of $\$$20532**

In [49]:
pred = grid_model.predict(X_test)

In [50]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [51]:
MAE = mean_absolute_error (y_test, pred)
MSE = mean_squared_error (y_test, pred)
RMSE = np.sqrt(MSE)

MAE, RMSE

(14195.354900562168, 20558.508566893164)

In [54]:
RMSE / np.mean(df['SalePrice'])

0.1136987941325648

## Great work!

----